In [5]:
#divergencias
import util
from backtesting import Backtest
import estrategias as est

#lista = lista= util.lista_de_monedas ()

lista = ['ETHUSDT','BCHUSDT','XRPUSDT','ADAUSDT','FILUSDT','MATICUSDT','OPUSDT','FETUSDT','BTCUSDT'] # lista acotada

lista=['BTCUSDT']

timeframe = '1h'

###
razon_max = 0.0
razon = 0.0
suma = 0.0
trades = 0
length_sma_max = 0
order_max = 0
length_adx_max = 0
for length_sma in [50]: #range(10,300,20):
    print(f'length_sma: {length_sma}')    
    for order in [550]: #range(200,3000,50):
        print(f'order: {order}')    
        for length_adx in [90]: #range(270,300,10):
            print(f'length_adx: {length_adx}')   
            try:
                suma = 0.0
                trades = 0
                razon = 0.0
                for symbol in lista:
                    try:                
                        #data = est.estrategia_adx (symbol,timeframe,length_sma,order,length_adx,start_date='2023-09-01')
                        data = est.estrategia_martillo (symbol,timeframe='15m',start_date='2024-10-01')
                        bt = Backtest(data,util.backtesting_config,cash=1000000,exclusive_orders=True)
                        output = bt.run()
                        if len(lista)==1:
                            bt.plot(filename="graficos/"+symbol)
                        print(f"Symbol: {symbol} - Return [%]: {round(output['Return [%]'],2)} - # Trades: {output['# Trades']} - Win Rate [%]: {output['Win Rate [%]']}")
                        suma=suma+output['Return [%]']
                        trades=trades+output['# Trades']
                    except Exception as e:
                        print(f"Error en {symbol}: {e}")
                if trades>0:
                    razon = suma/trades
                if razon > razon_max:
                    razon_max = razon
                    length_sma_max = length_sma
                    order_max = order
                    length_adx_max = length_adx
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
            except Exception as e:
                print(f"Error en {symbol}: {e}")
                print(f"Suma total: {suma} - Trades total: {trades} - razón: {razon} - order: {order} - length_adx: {length_adx}")
                print(f"razon_max: {razon_max} - length_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")
print(f"razon_max: {razon_max} - length_sma_max: {length_sma_max} - order_max: {order_max} - length_adx_max: {length_adx_max}")

length_sma: 50
order: 550
length_adx: 90
Error: 403 - <html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>

Error en BTCUSDT: 'Volume'
Error en BTCUSDT: OHLC `data` is empty
Suma total: 0.0 - Trades total: 0 - razón: 0.0 - order: 550 - length_adx: 90
razon_max: 0.0 - length_sma_max: 0 - order_max: 0 - length_adx_max: 0
razon_max: 0.0 - length_sma_max: 0 - order_max: 0 - length_adx_max: 0
